In [1]:
#Import Libraries

import logging
import os
import random
import sys
from dataclasses import dataclass, field
from sklearn.metrics import confusion_matrix
!pip install transformers
from transformers import BertTokenizer, BertModel, BertConfig
import torch
import torch.nn as nn
import warnings
import numpy as np
warnings.filterwarnings('ignore')


Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [2]:
from google.colab import drive
drive.mount('/content/drive', force_remount=True)
root_dir = "/content/drive/My Drive/DFKI"
os.chdir(root_dir)
from utils.dataset_transformer import ReviewsDataset
from utils.model_transformers import BertClassifier

Mounted at /content/drive


In [3]:
BATCH_SIZE = 256
EPOCHS = 100
STEP_SIZE = 0.003
SENTENCE_SIZE = 80
CLASSES = 5
MODEL = 'bert-base-cased'
DATA_LEN = 205000
FILE_NAME = 'sentiment_data.csv'

In [4]:
# Setting GPU as the default device for computation
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

In [5]:
#Initializing Model, loss function and optimizer
model = BertClassifier(num_classes = CLASSES,  model_name = MODEL, dropout = 0.25).to(device)
criterion = nn.CrossEntropyLoss().cuda()
optimizer = torch.optim.Adam(model.parameters(), lr=STEP_SIZE)

Some weights of the model checkpoint at bert-base-cased were not used when initializing BertModel: ['cls.seq_relationship.weight', 'cls.predictions.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.decoder.weight', 'cls.predictions.transform.dense.bias', 'cls.seq_relationship.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.LayerNorm.bias']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


In [ ]:
def train_model(model, optimizer, criterion):
  # Args
    acc = 0   
    for epoch in range(EPOCHS): 
        print (epoch)
        epoch_loss = 0
        test_predictions = []
        test_truth = []
        #Initializing Custom Pytorch Dataset class
        dataset = ReviewsDataset(csv_file = FILE_NAME, chunk_size = BATCH_SIZE, len_dataset = DATA_LEN, max_seq_len = SENTENCE_SIZE)
        train_batches = int(len(dataset) * 0.9)

        for i in range(len(dataset)): 
          vectors, targets = dataset[i]
          embeddings = vectors.to(device)

          mask = vectors['attention_mask'].to(device)
          input_id = vectors['input_ids'].squeeze(1).to(device)

          labels = torch.from_numpy(np.array(targets)).long().to(device)
          if i < train_batches:
            
            #Training the model
            optimizer.zero_grad()
            outputs = model(input_id, mask)
            loss = criterion(outputs, labels.cuda())
            loss.backward()
            optimizer.step()
            epoch_loss += outputs.shape[0] * loss.item()
          else:
            #Saving test results
            outputs = model(input_id, mask)         
            _, predictions = torch.max(torch.FloatTensor(outputs.cpu()), 1)
            test_predictions.extend(predictions.tolist())
            test_truth.extend(labels.cpu().tolist())

        #Printing test results and saving the model if the current test results are better than before
        test_predictions = np.array(test_predictions)
        test_truth = np.array(test_truth)
        test_acc = (test_truth == test_predictions).sum()/len(test_truth) 
        print(test_acc)
        if test_acc > acc:
            acc = test_acc
            torch.save(model.state_dict(), 'model_sentiment.ckpt')
            
            test_truth[test_truth == 0] = 1
            test_truth[test_truth == 4] = 3
            test_predictions[test_predictions == 0] = 1
            test_predictions[test_predictions == 4] = 3
            print(confusion_matrix(test_truth, test_predictions))
            acc_general = ((test_predictions==test_truth).sum())/len(test_truth)
            print (acc_general)
          
        
            
    return model
  
train_model(model, optimizer, criterion)




0
0.201171875
[[  43   48 7993]
 [  19   23 4060]
 [  38   55 8201]]
0.403662109375
1
0.20439453125
[[  26  397 7661]
 [  13  178 3911]
 [  25  350 7919]]
0.396630859375
2
0.20751953125
[[  18  259 7807]
 [   6  130 3966]
 [  14  257 8023]]
0.398974609375
3
0.20576171875
4
0.20615234375
5
0.2056640625
6
0.2025390625
7
0.20341796875
8
0.2046875
9
